In [1]:
import itertools
import time
from CliffordAlgebras import Clifford
import DiracOperator
import numpy as np
from numba import njit
# Some dummy parameters for use when testing:
p = 3
q = 1
N = 5
g2 = -3.7
g4 = 1
# Get the clifford algebra for the model we are examining.
cliff = Clifford(p, q)
cliff.introduce()
odd_products = cliff.get_odd_gamma_products()

My type is: (3,1)

My matrices will be 4x4, and I have K0 dimension s = 6
The generators are:
Gamma_1 = 
[[ 1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j -0.+0.j]
 [ 0.+0.j  0.+0.j -0.+0.j -1.+0.j]]
Gamma_2 = 
[[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]]
Gamma_3 = 
[[ 0.+0.j  0.+0.j  0.+1.j  0.+0.j]
 [ 0.+0.j -0.+0.j  0.+0.j -0.-1.j]
 [ 0.-1.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+1.j  0.+0.j -0.+0.j]]
Gamma_4 = 
[[ 0.+0.j  0.+0.j  0.+0.j  0.+1.j]
 [-0.+0.j  0.+0.j -0.-1.j  0.+0.j]
 [ 0.+0.j  0.-1.j  0.+0.j  0.+0.j]
 [ 0.+1.j  0.+0.j -0.+0.j  0.+0.j]]


The chirality operator is:
[[0.+0.j 0.+0.j 0.+0.j 0.+1.j]
 [0.+0.j 0.+0.j 0.+1.j 0.+0.j]
 [0.+0.j 0.-1.j 0.+0.j 0.+0.j]
 [0.-1.j 0.+0.j 0.+0.j 0.+0.j]]


There are some import hyperparameters for the Metropolis-Hastings algorithm. Specifically,
in our implementation, the variable weightA is important, which is the step size between Dirac operators.
WeightA to be very small, so that the steps between Diracs is relatively small. There is a balance between having a small enough step size so that many Diracs are accepted and we "follow the valley downwards" and having a large enough step size so that we can escape local minima.

This value will change for each type, it seems to be a hyperparameter of this setup, i.e. requires tuning. weightA values for the types that give acceptance_rate/num_moves ~50% (this is what I understand to be a good thing to achieve)

(2,0) -> 1./np.power(cliff.matdim,3)/25
(1,3) -> 1./e-2/6 for size 10x10 - takes 30mins to do 40,000 moves.

In [2]:
# Initialise D: we set the weight A so we start with a much wide range of the parameter space

weightA = 1 / 10  # If we use too large a value here, then the proposed starting Dirac causes the weight action to have a large imaginary component. So start with it ~1
matdim = cliff.matdim
D = DiracOperator.random_Dirac_op(odd_products, N, weightA, cliff.matdim)
# D = D1


@njit(fastmath=True)
def runMonteCarlo(odd_products, D, g2, g4, matdim):
    weightA = 1e-2 / 7
    num_moves = 0
    acceptance_rate = 0
    epochs = 10
    chain_length = 1000
    for i in range(epochs):
        for j in range(chain_length):
            D, acceptance_rate = DiracOperator.update_Dirac(
                D, g2, g4, weightA, acceptance_rate, N, matdim, odd_products)
            num_moves += 1
        # Every 1000s steps, we investigate a little. We print the accepted rate and the action of the Dirac at this moment.
        print(acceptance_rate, num_moves, acceptance_rate / num_moves)
        print(DiracOperator.action(D, g2, g4))
        # np.linalg.eigvals(D)


"""
It would be helpful to store the last Dirac, so we can "continue" the simulation after it reaches 20,0000 steps. For instance, for the type (1,3), 20,000 steps doesnt seem to be enough. 40,0000 seems to settle in a well.
But to have the thermalisation time be ~1%, then we need to continue the simulation for much longer than that.
"""

# We can then "initialise" the Dirac with the last Dirac operator from the simulation if we want to continnue the simulation.
D1 = D


/Users/pauldruce/.pyenv/versions/3.7.7/lib/python3.7/site-packages/numba/core/ir_utils.py:2031: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'odd_products' of function 'random_Dirac_op'.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "DiracOperator.py", line 68:
@njit(parallel=True, fastmath=True)
def random_Dirac_op(odd_products, N, weightA, matdim):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))


In [4]:
# Test it works and see how long it takes

%time runMonteCarlo(odd_products, D, g2, g4, matdim)

821 1000 0.821
-267.91866037156103
1699 2000 0.8495
-336.45639118100274
2605 3000 0.8683333333333333
-338.1316828846166
3497 4000 0.87425
-339.52940358698083
4395 5000 0.879
-337.1645506570599
5294 6000 0.8823333333333333
-333.8895190410342
6190 7000 0.8842857142857142
-337.11480113392366
7092 8000 0.8865
-336.6917042666361
8007 9000 0.8896666666666667
-339.04337187372477
8923 10000 0.8923
-340.12742224370646
CPU times: user 46.8 s, sys: 1min 11s, total: 1min 58s
Wall time: 52.3 s


In [5]:
# Profile it
%prun runMonteCarlo(odd_products, D, g2, g4, matdim)

848 1000 0.848
-301.65457214156334
1732 2000 0.866
-329.41753439701637
2637 3000 0.879
-338.66568206215635
3552 4000 0.888
-339.7451559117835
4462 5000 0.8924
-337.58570969856345
5365 6000 0.8941666666666667
-338.1063277385116
6272 7000 0.896
-334.6568717931576
7180 8000 0.8975
-335.3128305116874
8086 9000 0.8984444444444445
-339.368422899226
9003 10000 0.9003
-340.71479422712184
 

         1484 function calls (1466 primitive calls) in 52.243 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1   52.240   52.240   52.243   52.243 <ipython-input-2-5eba9ae8b0e9>:9(runMonteCarlo)
       90    0.001    0.000    0.001    0.000 socket.py:342(send)
       80    0.000    0.000    0.003    0.000 iostream.py:386(write)
       90    0.000    0.000    0.002    0.000 iostream.py:197(schedule)
       90    0.000    0.000    0.000    0.000 threading.py:1092(is_alive)
       80    0.000    0.000    0.000    0.000 iostream.py:310(_is_master_process)
       90    0.000    0.000    0.000    0.000 {method 'acquire' of '_thread.lock' objects}
       80    0.000    0.000    0.000    0.000 iostream.py:323(_schedule_flush)
       90    0.000    0.000    0.000    0.000 threading.py:1050(_wait_for_tstate_lock)
       90    0.000    0.000    0.000    0.000 iostream.py:93(_event_pipe)
        9    0.000    0.000    0.000   

In [8]:
%load_ext line_profiler

In [11]:
# Profile it
 runMonteCarlo(odd_products, D, g2, g4, matdim)

821 1000 0.821
-282.2733852259223
1708 2000 0.854
-335.4615902727121
2598 3000 0.866
-334.53251904087597
3502 4000 0.8755
-339.78751494096366
4402 5000 0.8804
-339.9971988078137
5308 6000 0.8846666666666667
-336.1104997663529
6203 7000 0.8861428571428571
-337.0768581853198
7104 8000 0.888
-331.40436871663553
8025 9000 0.8916666666666667
-335.81635970418597
8938 10000 0.8938
-341.2391831535927


Timer unit: 1e-06 s

Total time: 0 s
File: /Users/pauldruce/Documents/GitHub/RandomNCGinPython/DiracOperator.py
Function: update_Dirac at line 119

Line #      Hits         Time  Per Hit   % Time  Line Contents
   119                                           @njit(fastmath=True)
   120                                           def update_Dirac(old_D, g2, g4, weightA, acceptance_rate, N, matdim, odd_products):
   121                                               """ Updates the Dirac operator according to the Metropolis-Hastings algorithm
   122                                           
   123                                               This function inputs the old Dirac operator, the action coupling constants, weightA which dictates the steps size between the old and new Dirac operators, and a variable to calculate the acceptance_rate. This function could be done smoother I'm sure. But it seems to work.
   124                                           
   125                       